<a href="https://colab.research.google.com/github/jeffufpost/scattered-thoughts/blob/master/2023_08_16_IHME_Tech_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# "IHME Tech Assessment - Future Health Scenarios"
> "Research, Future Health Scenarios - IHME"

- toc: true
- badges: true
- comments: true
- categories: [epidemiology, COVID-19]
- image: images/kalman_filter.jpg

![](my_icons/kalman_filter.jpg "https://fr.mathworks.com/videos/understanding-kalman-filters-part-3-optimal-state-estimator--1490710645421.html")

In [ ]:
#collapse_hide
!pip install plotly
import pandas as pd
import numpy as np
import datetime
import math

from scipy import signal

import plotly.graph_objects as go
import plotly.express as px

from scipy.stats import expon
from scipy.stats import gamma
from scipy.stats import weibull_min

from numpy.random import default_rng
rng = default_rng()

# Let's build a numerical solution
def seir_model(init, parms, days):
    S_0, E_0, I_0, R_0 = init
    Epd, Ipd, Rpd = [0], [0], [0]
    S, E, I, R = [S_0], [E_0], [I_0], [R_0]
    dt=0.1
    t = np.linspace(0,days,int(days/dt))
    sigma, beta, gam = parms
    for _ in t[1:]:
        next_S = S[-1] - beta*S[-1]*I[-1]*dt
        Epd.append(beta*S[-1]*I[-1]*dt)
        next_E = E[-1] + (beta*S[-1]*I[-1] - sigma*E[-1])*dt
        Ipd.append(sigma*E[-1]*dt)
        next_I = I[-1] + (sigma*E[-1] - gam*I[-1])*dt
        Rpd.append(gam*I[-1]*dt)
        next_R = R[-1] + (gam*I[-1])*dt
        S.append(next_S)
        E.append(next_E)
        I.append(next_I)
        R.append(next_R)
    return np.stack([S, E, I, R, Epd, Ipd, Rpd]).T

     |████████████████████████████████| 12.9MB 313kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


## Convolution

### Convolution to get $I_{pd}$

We have seen that daily new infectious individuals is given by:

$$I_{pd}[j] = \sum_{n_L=0}^{M_L-1}h_L[n_L]~E_{pd}[j-n_L]= h_L[j]\circledast E_{pd}[j]$$

Where $h_L[j]$ describes the distribution of $T_{Latent}$.

### Convolution to get $R_{pd}$

Similarly, we have seen that daily new recovered individuals (deaths + recoveries in fact) is given by:

$$R_{pd}[j] = \sum_{n_I=0}^{M_I-1}h_I[n_I]~I_{pd}[j-n_I]= h_I[j]\circledast I_{pd}[j]$$

Where $h_I[j]$ describes the distribution of $T_{Infectious}$.

## Finding $h_L[j]$ and $h_I[j]$

* $h_L[j]$ is simply the probability of an individual having a latent period of j days
* $h_I[j]$ is similarly the probability of an individual having an infectious period of j days  